<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [25]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, PReLU
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, Nadam
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Normalizer

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [26]:
# load the dataset
df = pd.read_csv('churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [27]:
df.shape

(7043, 21)

In [28]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [29]:
#drop high cardinality customerID column that doesn't provide useful information
df = df.drop(columns='customerID')

In [30]:
df['OnlineBackup'].unique

<bound method Series.unique of 0       Yes
1        No
2       Yes
3        No
4        No
       ... 
7038     No
7039    Yes
7040     No
7041     No
7042     No
Name: OnlineBackup, Length: 7043, dtype: object>

In [31]:
#change values to binary if they are yes or no // male or female
df['Churn'] = df['Churn'].replace(regex='Yes', value=1)
df['Churn'] = df['Churn'].replace(regex='No', value=0)

df['gender'] = df['gender'].replace(regex='Female', value=1)
df['gender'] = df['gender'].replace(regex='Male', value=0)

df['Partner'] = df['Partner'].replace(regex='Yes', value=1)
df['Partner'] = df['Partner'].replace(regex='No', value=0)

df['Dependents'] = df['Dependents'].replace(regex='Yes', value=1)
df['Dependents'] = df['Dependents'].replace(regex='No', value=0)

df['PhoneService'] = df['PhoneService'].replace(regex='Yes', value=1)
df['PhoneService'] = df['PhoneService'].replace(regex='No', value=0)

df['OnlineSecurity'] = df['OnlineSecurity'].replace(regex='Yes', value=1)
df['OnlineSecurity'] = df['OnlineSecurity'].replace(regex='No', value=0)

df['OnlineBackup'] = df['OnlineBackup'].replace(regex='Yes', value=1)
df['OnlineBackup'] = df['OnlineBackup'].replace(regex='No', value=0)

df['DeviceProtection'] = df['DeviceProtection'].replace(regex='Yes', value=1)
df['DeviceProtection'] = df['DeviceProtection'].replace(regex='No', value=0)

df['TechSupport'] = df['TechSupport'].replace(regex='Yes', value=1)
df['TechSupport'] = df['TechSupport'].replace(regex='No', value=0)

df['StreamingTV'] = df['StreamingTV'].replace(regex='Yes', value=1)
df['StreamingTV'] = df['StreamingTV'].replace(regex='No', value=0)

df['StreamingMovies'] = df['StreamingMovies'].replace(regex='Yes', value=1)
df['StreamingMovies'] = df['StreamingMovies'].replace(regex='No', value=0)

df['PaperlessBilling'] = df['PaperlessBilling'].replace(regex='Yes', value=1)
df['PaperlessBilling'] = df['PaperlessBilling'].replace(regex='No', value=0)

In [32]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,0,0,0,0,34,1,No,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,0,0,0,0,2,1,No,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,0,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,1,0,0,0,2,1,No,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [33]:
#now split the data into train and validation sets
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['Churn'], random_state=42)
print(train.shape, val.shape)

(5634, 20) (1409, 20)


In [34]:
# baseline
train['Churn'].value_counts(normalize=True)

0    0.734647
1    0.265353
Name: Churn, dtype: float64

In [35]:
#initalize target, normalize values
target = 'Churn'
y_train = train[target]
y_train.value_counts(normalize=True)

0    0.734647
1    0.265353
Name: Churn, dtype: float64

In [36]:
#intialize majoirty class
majority = y_train.mode()[0]
y_pred = [majority] * len(y_train)

#show accuracy, should be same as majoirty value
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.7346467873624423

Hyperparameter tuning

In [37]:
target = 'Churn'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]

In [38]:
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.fit_transform(X_val)

In [39]:
scaler = StandardScaler() #mean 0 std 1

X_train = scaler.fit_transform(X_train_encoded)
X_val = scaler.transform(X_val_encoded)

In [40]:
inputs = X_train.shape[1]
print(inputs)

19


In [46]:
# random seed 
seed = 44
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.0001):
    # create model
    model = Sequential()
    model.add(Dense(128, input_shape=(inputs,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = learning_rate, name='Adam'), metrics=['accuracy'])
    return model

# create model
opt_model = KerasClassifier(build_fn=create_model, verbose=0, epochs=20, batch_size=100,) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
learning_rate = [0.0005, 0.001, 0.01, 0.05, 0.1]
param_grid = dict(optimizer='optimizer')

# Create Grid Search
grid = GridSearchCV(estimator=opt_model, param_grid=param_g, n_jobs=-1, verbose=0, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)


# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

NameError: name 'param_g' is not defined

In [42]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8029779314994812 using {'batch_size': 80, 'epochs': 20}
Means: 0.7914425611495972, Stdev: 0.017133296866690243 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7962356209754944, Stdev: 0.009874507869615868 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7944586515426636, Stdev: 0.017831841259554664 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7980096220970154, Stdev: 0.010178072504219643 with: {'batch_size': 60, 'epochs': 20}
Means: 0.8029779314994812, Stdev: 0.016566872877082802 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7985394954681396, Stdev: 0.01755337171819784 with: {'batch_size': 100, 'epochs': 20}


In [43]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [100],
              'epochs': [20, 50, 75]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7960556387901306 using {'batch_size': 100, 'epochs': 20}
Means: 0.7960556387901306, Stdev: 0.018029028078914062 with: {'batch_size': 100, 'epochs': 20}
Means: 0.7935706973075867, Stdev: 0.014498153494470355 with: {'batch_size': 100, 'epochs': 50}
Means: 0.7859380841255188, Stdev: 0.016572207260742083 with: {'batch_size': 100, 'epochs': 75}


In [44]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))     
    model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.801204252243042 using {'optimizer': 'RMSprop'}
Means: 0.7909106492996216, Stdev: 0.018381620462559332 with: {'optimizer': 'SGD'}
Means: 0.801204252243042, Stdev: 0.014115296463568565 with: {'optimizer': 'RMSprop'}
Means: 0.7813277006149292, Stdev: 0.014509992333687573 with: {'optimizer': 'Adagrad'}
Means: 0.5884156107902527, Stdev: 0.1163989085174854 with: {'optimizer': 'Adadelta'}
Means: 0.7985402703285217, Stdev: 0.016827226480594194 with: {'optimizer': 'Adam'}
Means: 0.7983632802963256, Stdev: 0.015180287441161289 with: {'optimizer': 'Adamax'}
Means: 0.7985413789749145, Stdev: 0.014862478158451885 with: {'optimizer': 'Nadam'}


In [ ]:
def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(64, activation=activation, input_shape=(inputs,)))
    model.add(Dense(32, activation=activation))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, activation='sigmoid'))   
    optimizer = Nadam(learning_rate=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [45]:
def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(64, kernel_initializer=init_mode, activation='softplus', input_shape=(inputs,)))
    model.add(Dense(32, kernel_initializer=init_mode, activation='softplus'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))   
    optimizer = Nadam(learning_rate=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8051106333732605 using {'init_mode': 'glorot_uniform'}
Means: 0.8020914196968079, Stdev: 0.016780283835322297 with: {'init_mode': 'uniform'}
Means: 0.8035125136375427, Stdev: 0.012980455970556882 with: {'init_mode': 'lecun_uniform'}
Means: 0.8031563282012939, Stdev: 0.01727436616749308 with: {'init_mode': 'normal'}
Means: 0.8001409053802491, Stdev: 0.011715423999340432 with: {'init_mode': 'zero'}
Means: 0.8044007897377015, Stdev: 0.013076296185022852 with: {'init_mode': 'glorot_normal'}
Means: 0.8051106333732605, Stdev: 0.013815964638099528 with: {'init_mode': 'glorot_uniform'}
Means: 0.8022679328918457, Stdev: 0.016254692911849517 with: {'init_mode': 'he_normal'}
Means: 0.8035114288330079, Stdev: 0.012779537254720334 with: {'init_mode': 'he_uniform'}


In [ ]:

from tensorflow.keras.constraints import MaxNorm


def create_model(dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(64, kernel_initializer='glorot_uniform', activation='softplus', kernel_constraint=MaxNorm(weight_constraint), input_shape=(inputs,)))
    model.add(Dense(32, kernel_initializer='glorot_uniform', kernel_constraint=MaxNorm(weight_constraint), activation='softplus'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='glorot_uniform', kernel_constraint=MaxNorm(weight_constraint), activation='sigmoid'))   
    optimizer = Nadam(learning_rate=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [ ]:
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, kernel_initializer='glorot_uniform', activation='softplus', kernel_constraint=MaxNorm(1), input_shape=(inputs,)))
    model.add(Dense(neurons, kernel_initializer='glorot_uniform', kernel_constraint=MaxNorm(1), activation='softplus'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(1, kernel_initializer='glorot_uniform', kernel_constraint=MaxNorm(1), activation='sigmoid'))   
    optimizer = Nadam(learning_rate=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=100, verbose=0) #model same eah time, pass in kerasclassifier then do parameter grid search

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
neurons = [1, 5, 10, 25, 50, 100]
param_grid = dict(neurons=neurons)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?